# 실전 인터랙티브 데이터 시각화

분석할 데이터를 읽고, Bokeh 라이브러리를 사용하여 다양한 인터랙티브 시각화 만들기

## 대상 데이터








2013년부터 2018년까지 한국과 EU 국가들 간의 무역과 관련된 정보를 포함하고 있는 데이터를 분석 대상으로 한다.

데이터 출처: https://www.kaggle.com/strikergene/south-korea-eu-trade-20132018

---


[참고] 컬럼 정보: year, country, export_weight, import_weight, export_amount, import_amount, trade_balance, HS_code, description


## 데이터 분석 목적 및 시각화 기법



> **분석 목적**
    
*   연도별 수출, 수입 증감 추세 파악 
*   무역 불균형 현상 파악
*   수출, 수입 품목 파악

  위 세 항목들을 통해 우리나라와 EU 국가들 간 무역 현황을 파악하고, 활발한   무역과 교류를 도모하기 위한 방안에 대해 생각해보는 것을 분석 목적으로 한다. 

> **시각화 기법**

*   Bar Plot  
*   Pi Chart
*   Line Plot


## 데이터 파일 읽기

In [ ]:
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
output_notebook()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/data/Korea_EU_trade_data.csv")
data

,Unnamed: 0,year,country,export_weight,import_weight,export_amount,import_amount,trade_balance,HS_code,description
0,1,2013,Slovakia,0.0,0.0,0,0,0,1,LIVE ANIMALS
1,2,2013,Austria,0.0,0.1,1,6,-5,1,LIVE ANIMALS
2,3,2013,Romania,0.0,0.2,1,2,-2,1,LIVE ANIMALS
3,4,2013,Poland,0.0,0.2,1,5,-4,1,LIVE ANIMALS
4,5,2013,Netherland,1.0,29.4,106,1511,-1404,1,LIVE ANIMALS
...,...,...,...,...,...,...,...,...,...,...
13666,13667,2018,Germany,0.0,443.6,0,0,0,99,OTHER ITEMS
13667,13668,2018,Czech Republic,0.0,63.8,0,0,0,99,OTHER ITEMS
13668,13669,2018,Bulgaria,0.0,3.2,0,0,0,99,OTHER ITEMS
13669,13670,2018,Belgium,0.0,27.5,0,0,0,99,OTHER ITEMS


## 인터랙티브 데이터 시각화

In [ ]:
# 모듈 불러오기
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6, GnBu6
from bokeh.layouts import row
from bokeh.transform import cumsum
from bokeh.layouts import gridplot
from bokeh.models.tools import HoverTool

In [ ]:
# 변수 정의
years = ['2013', '2014', '2015', '2016', '2017', '2018']
exp = data['export_amount'].groupby(data['year']).sum()
exp = np.asarray(exp, dtype=np.int64)
exw = data['export_weight'].groupby(data['year']).sum()
exw = np.asarray(exw, dtype=np.int64)
imp = data['import_amount'].groupby(data['year']).sum()
imp = np.asarray(imp, dtype=np.int64)
imw = data['import_weight'].groupby(data['year']).sum()
imw = np.asarray(imw, dtype=np.int64)
tblc = data['trade_balance'].groupby(data['year']).sum()

In [ ]:
# 연도별 수출량(p1), 수입량(p2) 추이 나타내는 바플롯 그리기 (cf. 연결된 상호작용 사용)

source = ColumnDataSource(data=dict(years=years, exp=exp, imp=imp, color=Spectral6))
tools = "box_select, lasso_select, help"

p1 = figure(x_range=years, y_range=(0, 63000000),
            plot_height=400, plot_width=550,
            tools=tools, title="Export amounts of Korea-EU by year")
p1.vbar(x='years', top='exp', width=0.7, color='color', legend_field='years', source=source)
p2 = figure(x_range=years, y_range=(0, 63000000),
            plot_height=400, plot_width=550,
            tools=tools, title="Import amounts of Korea-EU by year")
p2.vbar(x='years', top='imp', width=0.7, color='color', legend_field='years', source=source)

p1.y_range.start = 0
p1.x_range.range_padding = 0.1
p1.xaxis.major_label_orientation = 1
p1.xgrid.grid_line_color = None

p2.y_range.start = 0
p2.x_range.range_padding = 0.1
p2.xaxis.major_label_orientation = 1
p2.xgrid.grid_line_color = None

In [ ]:
# Trade Imbalance 나타내는 파이차트(p3) 그리기 (cf. hover 옵션 사용)

tblc_data = pd.Series(tblc).reset_index(name='trade_balance').rename(columns={'index':'year'})
tblc_data['color'] = GnBu6
tblc_data['angle'] = tblc_data['trade_balance']/tblc_data['trade_balance'].sum() * 2 * np.pi
p3 = figure(plot_height=400, plot_width=550,
           tools="hover", tooltips="@year: @trade_balance",
           title="Trade Imbalance")
p3.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='year', source=tblc_data)
p3.axis.axis_label=None
p3.axis.visible=False
p3.grid.grid_line_color=None

In [ ]:
# 연도별(index), 단위별(amount, weight) 수출, 수입 나타내는 라인플롯 그리기 (cf. hover 옵션 사용)

p4 = figure(plot_width=550, plot_height=400,
            title="Comparison of Export & Import")
p4.line(exp.cumsum(), exw.cumsum(), color="red",
        alpha=0.6, line_width=3, legend_label="export")
p4.line(imp.cumsum(), imw.cumsum(), color="blue",
        alpha=0.6, line_width=3, legend_label="import")
p4.xaxis.axis_label = "amount"
p4.yaxis.axis_label = "weight"
p4.legend.location = "bottom_right"

cr1 = p4.circle(exp.cumsum(), exw.cumsum(), size=20,
            fill_color="gray", hover_fill_color="pink",
            fill_alpha=0.3, hover_alpha=0.7,
            line_color=None, hover_line_color="white")
cr2 = p4.circle(imp.cumsum(), imw.cumsum(), size=20,
            fill_color="gray", hover_fill_color="skyblue",
            fill_alpha=0.3, hover_alpha=0.7,
            line_color=None, hover_line_color="white")

p4.add_tools(HoverTool(tooltips=[("index", "$index"), ("(x, y)", "($x, $y)")],
                       renderers=[cr1, cr2], mode="vline"))

In [ ]:
# Layout으로 한 번에 나타내기 by gridplot()

p = gridplot([[p1, p2], [p3, p4]])
show(p)

## 분석 결과




1.  연도별(2013~2018) 수출량(p1), 수입량(p2) 비교
*   모든 연도에 걸쳐 수출량보다 수입량이 더 많음
*   수출량, 수입량 모두 2018년에 급감

2.  무역 불균형(p3)
*   모든 연도에 걸쳐 무역 불균형 발생
*   2014년과 2015년의 무역 불균형이 약 50% 차지
*   2016년, 2017년, 2018년을 지나며 무역 불균형이 많이 줄어드는 추세.
    (단, 무역량이 급감했던 2018년은 분석에서 제외해야 할 것으로 판단됨.)
    2016년과 2017년의 무역 불균형 감소현상은 유의미한 결과라 할 수 있음.


3.  연도별, 단위별 수출, 수입 비교
*   수출의 경우 weigth가, 수입의 경우 amount가 더 많은 것을 보아,
    중공업 또는 첨단 산업 제품의 수출 때문일 것으로 추정.
*   1의 결과와 마찬가지로 2018년의 수입, 수출이 가장 저조함. 








